In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
from multiprocessing import Pool, Lock, Value
from time import sleep
import time
import pandas as pd
import requests
import re
import html

In [4]:
%%time
authors_id = [29369, 29442, 102994, 26149, 35952]
books_urls = []
books_ids = []

with webdriver.Firefox() as driver:
    for page in authors_id:
        driver.get("https://www.bookvoed.ru/author/books?id={}"
                   .format(page))
        clicked = []
        sleep(1)
        last_height = 0
        while True:
            while True:
                driver.execute_script(
                    "window.scrollTo(0, document.body.scrollHeight);")
                sleep(1)
                new_height = driver.execute_script(
                    "return document.body.scrollHeight")
                if new_height == last_height:
                    break
                last_height = new_height
            try:
                button = (driver.find_element_by_css_selector('a.ty'))
                if button is not None:
                    button = button.get_attribute('onclick')
                    if button in clicked:
                        print("end of page {}"
                              .format(page))
                        break
                    driver.execute_script(button + ';')
                    clicked.append(button)
                    sleep(0.5)
                    print('button')
            except:
                print("end of page {}"
                      .format(page))
                break
                
        content = driver.find_elements_by_css_selector('div.gf')
        for ids in content:
            books_urls.append('https://www.bookvoed.ru/book?id={}'
                              .format(ids.get_attribute("data-book")))
            books_ids.append(ids.get_attribute("data-book"))

button
end of page 29369
end of page 29442
end of page 102994
end of page 26149
end of page 35952
CPU times: user 3.63 s, sys: 171 ms, total: 3.8 s
Wall time: 1min 40s


In [5]:
print(len(books_urls))
for i in books_urls[:5]:
    print(i)

1773
https://www.bookvoed.ru/book?id=9340995
https://www.bookvoed.ru/book?id=9340993
https://www.bookvoed.ru/book?id=9689115
https://www.bookvoed.ru/book?id=8645468
https://www.bookvoed.ru/book?id=8867666


In [6]:
agelimit = {'sv nM': '18+', 'rv nM': '16+', 'qv nM': '12+', 'pv nM': '6+', 'ov nM': '0+'}

In [7]:
def extract_book_info(number):
    book_html = requests.get(books_urls[number]).text
    soup = BeautifulSoup(book_html, 'html.parser')
    desc = soup.find('div', class_='lw')
    if desc is not None:
        for nn in desc.find_all('div'):
            nn.replace_with(' ')
        desc = re.sub('\s+', ' ', desc.text)   
    table = soup.find('table', class_='tw')
    if table is not None:
        rows = table.find_all('tr')
        if rows is not None:
            data = [list(map(lambda x: x.text, row.find_all('td')))
                    for row in rows]
            book_info1 = dict(data)
    book_info = {}
    if len(book_info1) != 0:
        for old_key in book_info1:
            new_key = old_key[:-1]
            book_info[new_key] = book_info1[old_key]
        for lim in agelimit:
            if soup.find('div', class_=lim) is not None:
                agel = agelimit[lim]
    book_info['Возраст'] = agel
    book_info['ID']  = int(books_ids[number])
    f = soup.find('h1', itemprop='name')
    if f is not None:
        book_info['Название'] = f.text
    f = soup.find('img', class_='tf')
    if f is not None:
        book_info['Обложка'] = f.attrs['src']
    f = soup.find('div', class_='af')
    if f is not None:
        s = re.findall(r'\d{2}\.\d{2}%', f.attrs['style'])
        if len(s) != 0:
            book_info['Рейтинг'] = s[0]
    f = soup.find('a', class_='Ke Me ')
    if f is not None:
        s = re.findall(r'\d{1,7}', f.text)
        if len(s) != 0:
            book_info['Понравилось'] = int(s[0])
    f = soup.find('a', class_='Ke Le ff')
    if f is not None:
        s = re.findall(r'\d{1,7}', f.text)
        if len(s) != 0:
            book_info['В закладки'] = int(s[0])
    f = soup.find('a', class_='Ke Oe ')
    if f is not None:
        s = re.findall(r'\d{1,7}', f.text)
        if len(s) != 0:
            book_info['Не понравилось'] = int(s[0])
    f = soup.find('div', class_='Hu Wu')
    if f is not None:
        s = re.findall(r'\d{3,5} ....', f.text)
        if len(s) != 0:
            book_info['Цена'] = s[0]
    book_info['Описание'] = desc
    return book_info

In [8]:
%%time
mutex = Lock()
n_processed = Value('i', 0)


def func_wrapper(uid):
    res = extract_book_info(uid)
    with mutex:
        global n_processed
        n_processed.value += 1
        if n_processed.value % 10 == 0:
            print(f"\r{n_processed.value} objects are processed...", end='', flush=True)
    return res

with Pool(processes=10) as pool:
    res = pool.map(func_wrapper, range(len(books_urls)))

df = pd.DataFrame(res)
df.sort_values(by=['ID'], inplace=True)
with open('data/hw_3.csv', mode='w', encoding='utf-8') as f_csv:
    df.to_csv(f_csv, index=False)    

1770 objects are processed...CPU times: user 554 ms, sys: 218 ms, total: 771 ms
Wall time: 5min 44s


In [9]:
df.head(50)

,ID,ISBN,Автор,Авторы,В базе,В закладки,Возраст,Год,Издательство,Код,...,Понравилось,Производитель,Размеры,Рейтинг,Серия,Страниц,Тематика,Тираж,Формат,Цена
953,330658,978-5-699-20894-4,Донцова Дарья Аркадьевна,NaN,Э.ИД.Золушка в шоколаде,34,16+,2007,Эксмо,333104,...,7.0,NaN,"13,00 см x 20,00 см x 2,50 см",NaN,Иронический детектив,384,Отечественные,250000,84х108/32,NaN
1772,330661,978-5-699-20169-3,Емец Дмитрий Александрович,NaN,Э.Мефодий Буслаев-7.Лед и пламя Тартара,26,12+,2013,Эксмо,323903,...,359.0,NaN,"13,00 см x 20,00 см x 2,60 см",98.09%,Мефодий Буслаев,416,Для среднего школьного возраста,90100,205.00mm x 135.00mm x 25.00mm,NaN
952,330670,5-699-19930-6,Донцова Дарья Аркадьевна,NaN,Э.ИД.Каникулы в Простофилино,34,16+,2007,Эксмо,325231,...,5.0,NaN,"13,00 см x 20,00 см x 2,50 см",57.14%,Иронический детектив,384,Отечественные,250000,84х108/32,NaN
951,330918,978-5-699-21569-0,Донцова Дарья Аркадьевна,NaN,Э.ИД.Зимнее лето весны,2,16+,2007,Эксмо,343340,...,7.0,NaN,"13,00 см x 20,00 см x 2,60 см",66.67%,Иронический детектив,384,Отечественные,250000,84х108/32,NaN
950,330920,978-5-699-22452-4,Донцова Дарья Аркадьевна,NaN,Э.ИД.Личное дело Женщины-кошки,34,16+,2007,Эксмо,347336,...,4.0,NaN,"13,00 см x 20,00 см x 2,70 см",NaN,Иронический детектив,384,Отечественные,250000,84х108/32,NaN
949,330930,5-699-18855-X,Донцова Дарья Аркадьевна,NaN,Э.ПИ.Простые и вкус.рец.Дарьи Донцовой,2,12+,2006,Эксмо,306488,...,4.0,NaN,"21,80 см x 29,00 см x 1,80 см",46.67%,NaN,256,"Кулинария, напитки",50100,60х84/8,NaN
948,330943,978-5-699-21242-2,Донцова Дарья Аркадьевна,NaN,Э.ИД(м).Каникулы в Простофилино,34,16+,2007,Эксмо,337631,...,2.0,NaN,"10,90 см x 16,50 см x 2,00 см",NaN,Иронический детектив,384,Отечественные,370000,70х90/32,NaN
947,331009,978-5-699-23848-4,Донцова Дарья Аркадьевна,NaN,Э.Кулинарная книга лентяйки-2,34,12+,2007,Эксмо,357525,...,3.0,NaN,"21,50 см x 17,00 см x 1,80 см",54.55%,NaN,304,"Кулинария, напитки",150000,70х90/16,NaN
946,331010,5-699-19407-X,Донцова Дарья Аркадьевна,NaN,Э.ИД.Мачо чужой мечты,1,16+,2006,Эксмо,314219,...,9.0,NaN,"13,00 см x 20,00 см x 2,90 см",NaN,Иронический детектив,384,Отечественные,170000,84х108/32,NaN
945,331011,978-5-699-21101-2,Донцова Дарья Аркадьевна,NaN,"Э.ИД.Верхом на ""Титанике""",2,16+,2007,Эксмо,339651,...,5.0,NaN,"13,00 см x 20,00 см x 2,60 см",72.73%,Иронический детектив,384,Отечественные,190000,84х108/32,NaN
